In [ ]:
!pip3 install roboticstoolbox-python
!pip3 install pymoo

In [33]:
# Robotics library, help us to easily valid IK approach with ours, also we can use
# for computation of FK
import roboticstoolbox as rtb
from spatialmath import SE3

# load UR5 model
robot = rtb.models.DH.UR5()
print(robot)

# simple computation of Inverse Kinematics from defined X,Y,Z position
Tep = SE3.Trans(.5, .3, .5)
sol = robot.ik_LM(Tep)
print(sol[0])

# validate using Forward Kinematics predicted position
print(robot.fkine([-2.96837897, -0.29736756, -0.54961498,  1.43990795,  3.09161809,  1.95641203]))

DHRobot: UR5 (by Universal Robotics), 6 joints (RRRRRR), dynamics, standard DH parameters
┌─────┬─────────┬─────────┬────────┐
│ θⱼ  │   dⱼ    │   aⱼ    │   ⍺ⱼ   │
├─────┼─────────┼─────────┼────────┤
│  q1 │ 0.08946 │       0 │  90.0° │
│  q2 │       0 │  -0.425 │   0.0° │
│  q3 │       0 │ -0.3922 │   0.0° │
│  q4 │  0.1091 │       0 │  90.0° │
│  q5 │ 0.09465 │       0 │ -90.0° │
│  q6 │  0.0823 │       0 │   0.0° │
└─────┴─────────┴─────────┴────────┘

┌──┬──┐
└──┴──┘

┌──────┬───────┬─────┬─────┬─────┬──────┬─────┐
│ name │ q0    │ q1  │ q2  │ q3  │ q4   │ q5  │
├──────┼───────┼─────┼─────┼─────┼──────┼─────┤
│   qr │  180° │  0° │  0° │  0° │  90° │  0° │
│   qz │  0°   │  0° │  0° │  0° │  0°  │  0° │
└──────┴───────┴─────┴─────┴─────┴──────┴─────┘

[-2.78955536 -0.86985901  0.8625156   1.57814941 -1.57079253  1.21875605]
   0.2064   -0.955     0.2129    0.6028    
   0.01703  -0.2141   -0.9767    0.1328    
   0.9783    0.2052   -0.02791   0.4271    
   0         0         0   

In [ ]:
from pymoo.algorithms.soo.nonconvex.pso import PSO
from pymoo.core.problem import Problem
from pymoo.optimize import minimize

import roboticstoolbox as rtb
import numpy as np

# Define the target end-effector pose
# we can start with X,Y,Z position only
target_pose = np.array([.5, .3, .7])

# load D-H parameters from UR5 kinematics structure
robot = rtb.models.DH.UR5()

# define IK problem for robotic ARM
class IK_Problem(Problem):
  def __init__(self):
      super().__init__(
          n_var=6,
          xl=np.array([-np.pi,-np.pi,-np.pi,-np.pi,-np.pi,-np.pi]),
          xu=np.array([np.pi,np.pi,np.pi,np.pi,np.pi,np.pi]),
          n_obj=1
      )

  def _evaluate(self, x, out, *args, **kwargs):
      res = []
      for q in x:
        # Forward kinematics to calculate the end-effector position based on joint angles x
        A = robot.fkine(np.array(q))
        # simple euclidian distance between pred_point and des_point
        distance = np.linalg.norm(A.t - target_pose)
        res.append(distance)

      out["F"] = np.array(res)

# set up optimization algo
problem = IK_Problem()
algorithm = PSO(restarts=20, pop_size = 100, restart_from_best=True)
res = minimize(problem, algorithm, ('n_gen', 100), seed=50, verbose=True )

print(f"Best solution found: \nX = {res.X}\nF = {res.F}")

xyz = robot.fkine(np.array(res.X))
print(f"Solution val: {xyz.t}")

n_gen  |  n_eval  |    f     |    S    |    w    |    c1    |    c2    |     f_avg     |     f_min    
     1 |      100 |        - |       - |  0.9000 |  2.00000 |  2.00000 |  0.9806153755 |  0.2348887975
     2 |      200 |  0.30102 |       2 |  0.5932 |  2.03996 |  1.96004 |  0.8797053027 |  0.1365708701
     3 |      300 |  0.14312 |       3 |  0.4917 |  2.03942 |  1.97423 |  0.7036055343 |  0.1041713637
     4 |      400 |  0.07280 |       3 |  0.4462 |  2.03173 |  1.99513 |  0.5734842042 |  0.1041713637
     5 |      500 |  0.02418 |       3 |  0.4152 |  2.01778 |  2.01067 |  0.4729695783 |  0.0353919928
     6 |      600 |  0.05186 |       3 |  0.4328 |  2.00348 |  2.01907 |  0.4121666235 |  0.0037228112
     7 |      700 |  0.02015 |       3 |  0.4126 |  1.99243 |  2.03726 |  0.3567135038 |  0.0037228112
     8 |      800 |  0.01031 |       3 |  0.4065 |  1.97809 |  2.04448 |  0.3170963610 |  0.0037228112
     9 |      900 |  0.00814 |       3 |  0.4051 |  1.96751 |  2.05394 | 